In [38]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [39]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1) #input -? OUtput? RF 28 > 26
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)  # 28 > 26
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.conv4 = nn.Conv2d(128, 256, 3, padding=1)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.conv5 = nn.Conv2d(256, 512, 3)
        self.conv6 = nn.Conv2d(512, 1024, 3) # o
        self.conv7 = nn.Conv2d(1024, 10, 3) # o

    def forward(self, x, y):  # 
        x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x)))))
        x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x)))))
        x = F.relu(self.conv6(F.relu(self.conv5(x))))  # o
        x = (self.conv7(x))  # F.relu  # o
        x = x.view(-1, 10)
        # print('x is ', x)
        # print('log x is ', F.log_softmax(x).shape,)
        x = F.log_softmax(x)
        x_for_sum = torch.argmax(x, dim=1)
        sum_x_y = x_for_sum + y
        # print('both sare ', sum_x_y)

        # y_tensor = torch.tensor([y])
        sum_x_y_encoded = F.one_hot(sum_x_y, num_classes=max_rand_no*2)

        # output[1].argmax()

        return x, sum_x_y_encoded# , rand
		

In [40]:
# a = torch.Tensor([2,3,4,5])
# a.reshape(1)

In [41]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
# summary(model, input_size=(1, 28, 28))

In [42]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [43]:
import random
import sys
max_rand_no = 10


from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()

        randomlist = []
        for i in range(0,data.size()[0]):
          rand_no = random.randint(0,max_rand_no)
          randomlist.append(rand_no)
        randomtensor = torch.as_tensor(randomlist)
        randomtensor = randomtensor.cuda()

        output, sum_rand_image = model(data, randomtensor)  # sum_randomtensor
        loss = F.cross_entropy(output, target)
        sum_rand_image = sum_rand_image.type(torch.FloatTensor)
        target_2 = (target+randomtensor)
        sum_rand_image = sum_rand_image.cuda()

        loss_2 = F.cross_entropy(sum_rand_image, target_2)
        loss = loss+loss_2
        # print('losses are ', loss, loss_2)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    loss_2 = 0
    correct = 0
    correct_2 = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            
            randomlist = []
            for i in range(0,data.size()[0]):
              rand_no = random.randint(0,max_rand_no)
              randomlist.append(rand_no)
            randomtensor = torch.as_tensor(randomlist)
            randomtensor = randomtensor.cuda()
            
            output, sum_rand_image = model(data, randomtensor)
            test_loss += F.cross_entropy(output, target, reduction='sum').item()  # sum up batch loss
            sum_rand_image = sum_rand_image.type(torch.FloatTensor)
            target_2 = (target+randomtensor)
            sum_rand_image = sum_rand_image.cuda()
            loss_2 = F.cross_entropy(sum_rand_image, target_2, reduction='sum').item()
            loss = test_loss+loss_2

            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

            pred_2 = sum_rand_image.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct_2 += pred_2.eq(target_2.view_as(pred_2)).sum().item()
    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [44]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 2):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)






  0%|          | 0/469 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.





loss=5.223395824432373 batch_id=0:   0%|          | 0/469 [00:00<?, ?it/s]




loss=5.223395824432373 batch_id=0:   0%|          | 1/469 [00:00<01:05,  7.16it/s]




loss=5.263501167297363 batch_id=1:   0%|          | 1/469 [00:00<01:05,  7.16it/s]




loss=5.231509208679199 batch_id=2:   0%|          | 1/469 [00:00<01:05,  7.16it/s]




loss=5.231509208679199 batch_id=2:   1%|          | 3/469 [00:00<00:55,  8.46it/s]




loss=5.22055721282959 batch_id=3:   1%|          | 3/469 [00:00<00:55,  8.46it/s] 




loss=5.261728286743164 batch_id=4:   1%|          | 3/469 [00:00<00:55,  8.46it/s]




loss=5.261728286743164 batch_id=4:   1%|          | 5/469 [00:00<00:46, 10.03it/s]




loss=5.300722122192383 batch_id=5:   1%|          | 5/469 [00:00<00:46, 1


Test set: Average loss: 0.0696, Accuracy: 9764/10000 (98%)

